In [1]:
#import os
#import numpy as np 
#import * #bring all module all at once 
#from collections import Counter  

# Import all other necessary libraries. Your code below ...
#import re
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.metrics import accuracy_score

In [8]:
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Function to create a dictionary of the most common words in emails
def make_Dictionary(root_dir):
    all_words = []
    # Get a list of file paths for all emails in the directory
    emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
    for mail in emails:
        with open(mail) as m:
            for line in m:
                words = line.split()
                all_words += words
    # Use Counter to count word frequencies in all emails
    dictionary = Counter(all_words)
    
    # Remove non-alphabetic tokens and single character tokens from the dictionary
    list_to_remove = list(dictionary.keys())
    for item in list_to_remove:
        if not item.isalpha():
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
            
    # Select the top 3000 most common words to use in the dictionary
    dictionary = dictionary.most_common(3000)
    return dictionary

# Function to extract features from emails based on the dictionary
def extract_features(mail_dir):
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    # Initialize the matrix to hold feature vectors and the array for labels
    features_matrix = np.zeros((len(files),3000))
    train_labels = np.zeros(len(files))
    docID = 0
    for fil in files:
        with open(fil) as fi:
            for i,line in enumerate(fi):
                # Only process the third line of each email (where content often starts)
                if i == 2:
                    words = line.split()
                    for word in words:
                        for i,d in enumerate(dictionary):
                            if d[0] == word:
                                # Increment the word count for this document in the feature matrix
                                features_matrix[docID,i] = words.count(word)
        # Set the label for this document (0 for normal, 1 for spam)
        train_labels[docID] = 0
        filepathTokens = fil.split('/')
        lastToken = filepathTokens[-1]
        if lastToken.startswith("spmsg"):
            train_labels[docID] = 1
        docID += 1
    return features_matrix, train_labels

# Directories containing the training and test email datasets
TRAIN_DIR = "./train-mails"
TEST_DIR = "./test-mails"

# Build the dictionary from the training dataset
dictionary = make_Dictionary(TRAIN_DIR)

print(f'reading and processing emails from TRAIN and TEST folders')
# Extract features and labels from the training and test datasets
features_matrix, labels = extract_features(TRAIN_DIR)
test_feature_matrix, test_labels = extract_features(TEST_DIR)

# Initialize the Gaussian Naive Bayes model
model = GaussianNB()

print(f'Training Model using Gaussian Naibe Bayes algorithm .....')
# Train the model using the training dataset
model.fit(features_matrix, labels)
print(f'Training completed')
# Predict the labels of the test dataset
predicted_labels = model.predict(test_feature_matrix)
print(f'testing trained model to predict Test Data labels')

# Print the accuracy of the model on the test dataset
print(f'Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels: \n{accuracy_score(test_labels, predicted_labels)}')
#print end of the program
print(f'\n\n{"="*15} End of the program {"="*15}')

reading and processing emails from TRAIN and TEST folders
Training Model using Gaussian Naibe Bayes algorithm .....
Training completed
testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels: 
0.9653846153846154


=============== End of the program ===============
